# Convex Optimization in Julia

## Madeleine Udell | ACC 2017

In [1]:
# initial package installation
Pkg.add("Convex")
Pkg.add("SCS")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Convex
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of SCS
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
# Make the Convex.jl module available
using Convex
using SCS # first order splitting conic solver [O'Donoghue et al., 2014]
set_default_solver(SCSSolver(verbose=0)) # could also use Gurobi, Mosek, CPLEX, ...

# Generate random problem data
m = 50;  n = 100
A = randn(m, n)
x♮ = sprand(n, 1, .5) # true (sparse nonnegative) parameter vector
noise = .1*randn(m)    # gaussian noise
b = A*x♮ + noise      # noisy linear observations

# Create a (column vector) variable of size n.
x = Variable(n)

# nonnegative elastic net with regularization
λ = 1
μ = 1
problem = minimize(norm(A * x - b)^2 + λ*norm(x)^2 + μ*norm(x, 1), 
                   x >= 0)

# Solve the problem by calling solve!
solve!(problem)

println("problem status is ", problem.status) # :Optimal, :Infeasible, :Unbounded etc.
println("optimal value is ", problem.optval)

problem status is Optimal
optimal value is 38.443579438332975


# Quick convex prototyping

## Variables

In [3]:
# Scalar variable
x = Variable()

Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [4]:
# (Column) vector variable
y = Variable(4)

Variable of
size: (4, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [5]:
# Matrix variable
Z = Variable(4, 4)

Variable of
size: (4, 4)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

# Expressions

Convex.jl allows you to use a [wide variety of functions](http://convexjl.readthedocs.org/en/latest/operations.html) on variables and on expressions to form new expressions.

In [6]:
x + 2x

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [7]:
e = y[1] + logdet(Z) + sqrt(x) + minimum(y)

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConcaveVexity()


### Examine the expression tree

In [8]:
e.children[2]

AbstractExpr with
head: logdet
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConcaveVexity()


# Constraints

A constraint is convex if convex combinations of feasible points are also feasible. Equivalently, feasible sets are convex sets.

In other words, convex constraints are of the form

* `convexExpr <= 0`
* `concaveExpr >= 0`
* `affineExpr == 0`

In [9]:
x <= 0

Constraint:
<= constraint
lhs: Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()
rhs: 0
vexity: Convex.AffineVexity()

In [10]:
x^2 <= sum(y)

Constraint:
<= constraint
lhs: AbstractExpr with
head: qol_elem
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: AbstractExpr with
head: sum
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

vexity: Convex.ConvexVexity()

In [11]:
M = Z 
for i = 1:length(y)
    M += rand(size(Z))*y[i]
end
M ⪰ 0

Constraint:
sdp constraint
expression: AbstractExpr with
head: +
size: (4, 4)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()



# Problems

In [12]:
x = Variable()
y = Variable(4)
objective = 2*x + 1 - sqrt(sum(y))
constraint = x >= maximum(y)
p = minimize(objective, constraint)

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
>= constraint
lhs: Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()
rhs: AbstractExpr with
head: maximum
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

vexity: Convex.ConvexVexity()
current status: not yet solved

In [13]:
# solve the problem
solve!(p)
p.status

:Optimal

In [14]:
x.value

0.2499283491590887

In [15]:
# can evaluate expressions directly
evaluate(objective)

1×1 Array{Float64,2}:
 0.500004

## Pass to solver

call a `MathProgBase` solver suited for your problem class

* see the [list of Convex.jl operations](http://convexjl.readthedocs.org/en/latest/operations.html) to find which cones you're using
* see the [list of solvers](http://www.juliaopt.org/) for an up-to-date list of solvers and which cones they support

to solve problem using a different solver, just import the solver package and pass the solver to the `solve!` method: eg

    using Mosek
    solve!(p, MosekSolver())

## Warmstart

In [16]:
# Generate random problem data
m = 100;  n = 200
A = randn(m, n)
x♮ = sprand(n, 1, .5) # true (sparse nonnegative) parameter vector
noise = .1*randn(m)    # gaussian noise
b = A*x♮ + noise      # noisy linear observations

# Create a (column vector) variable of size n.
x = Variable(n)

# nonnegative elastic net with regularization
λ = 1
μ = 1
problem = minimize(norm(A * x - b)^2 + λ*norm(x)^2 + μ*norm(x, 1), 
                   x >= 0)
@time solve!(problem)
λ = 1.5
@time solve!(problem, warmstart = true)

  0.157229 seconds (20.56 k allocations: 9.718 MB)
  0.194900 seconds (67.05 k allocations: 11.565 MB, 5.63% gc time)


# DCP examples

In [17]:
# affine
x = Variable(4)
y = Variable(2)
sum(x) + y[2]

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [18]:
2*maximum(x) + 4*sum(y) - sqrt(y[1] + x[1]) - 7 * minimum(x[2:4])

AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()


In [19]:
# not dcp compliant
log(x) + x.^2

AbstractExpr with
head: +
size: (4, 1)
sign: Convex.NoSign()
vexity: Convex.NotDcp()


In [20]:
# $f$ is convex increasing and $g$ is convex
square(pos(x))

AbstractExpr with
head: qol_elem
size: (4, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()


In [21]:
# $f$ is convex decreasing and $g$ is concave 
invpos(sqrt(x))

AbstractExpr with
head: qol_elem
size: (4, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()


In [22]:
# $f$ is concave increasing and $g$ is concave 
sqrt(sqrt(x))

AbstractExpr with
head: geomean
size: (4, 1)
sign: Convex.Positive()
vexity: Convex.ConcaveVexity()
